In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
train_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/train_hypothesis_evidences.csv')
train_df

,hypothesis,evidences,label
0,All the primary trial participants do not rece...,"[""INTERVENTION 1: "", "" Diagnostic (FLT PET)"",...",Contradiction
1,"Patients with Platelet count over 100,000/mm¬¨...","["" PATIENT CHARACTERISTICS:"", "" ANC 1,500/m...",Contradiction
2,Heart-related adverse events were recorded in ...,"[""Adverse Events 1:"", "" Supraventricular tach...",Entailment
3,Adult Patients with histologic confirmation of...,"[""Inclusion Criteria:"", "" Patients with histo...",Contradiction
4,Laser Therapy is in each cohort of the primary...,"[""INTERVENTION 1: "", "" Laser Therapy Alone"", ...",Contradiction
...,...,...,...
1695,"Adequate blood, kidney, and hepatic function a...","[""Inclusion Criteria:"", "" Postmenopausal wome...",Entailment
1696,The Ridaforolimus + Dalotuzumab + Exemestane g...,"[""Outcome Measurement: "", "" 1. Progression-fr...",Contradiction
1697,The only difference between the interventions ...,"[""INTERVENTION 1: "", "" Prone"", ""Prone positio...",Entailment
1698,Patients must have a white blood cell count ab...,"["" WBC > 1,500/mm\u00b3""]",Entailment


In [3]:
hypothesis_lst=train_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=train_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=train_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

1700

In [4]:
import torch
import numpy as np
import random
import math
from torch.utils.data import Dataset

class InputSequence(Dataset):
    def __init__(self, tok, l_text, l_text2, l_label=None, batch_size=64, gpu=True):
        self.data_len = len(l_text)
        self.data_idx = list(range(self.data_len))
        self.texts = tok(l_text, l_text2, padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label = np.array(l_label) if l_label is not None else None  # Initialize labels only if provided
        self.batch_size = batch_size
        self.gpu = gpu
        print('Tokenization done')

    def on_epoch_end(self):
        random.shuffle(self.data_idx)

    def __getitem__(self, i):
        start = i * self.batch_size
        batch_idx = self.data_idx[start:min(start + self.batch_size, self.data_len)]

        # Convert tensors to CUDA if GPU is enabled
        return_texts = {k: self.texts[k][batch_idx].cuda() if self.gpu else self.texts[k][batch_idx] for k in self.texts}

        if self.l_label is not None:  # Check if labels were provided
            return_labels = torch.from_numpy(self.l_label[batch_idx].astype(np.int64))
            if self.gpu:
                return_labels = return_labels.cuda()
            return return_texts, return_labels
        else:
            return return_texts  # Only return texts if no labels are available

    def __len__(self):
        return math.ceil(self.data_len / self.batch_size)


In [5]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

text_tok = AutoTokenizer.from_pretrained("MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c")
text_clf = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c")
text_clf.num_parameters()

tokenizer_config.json:   0%|          | 0.00/500 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/568M [00:00<?, ?B/s]

141896450

In [7]:
training_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)
len(training_data)

Tokenization done


27

In [8]:
dev_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/dev_hypothesis_evidences.csv')
dev_df

,hypothesis,evidences,label
0,there is a 13.2% difference between the result...,"[""Outcome Measurement: "", "" Event-free Surviv...",Contradiction
1,Patients with significantly elevated ejection ...,"["" Cardiac left ventricular function with res...",Contradiction
2,a significant number of the participants in th...,"["" Enterocolitis 1/167 (0.60%)"", "" Enterocol...",Contradiction
3,the primary trial does not report the PFS or o...,"[""Outcome Measurement: "", "" Local Control Usi...",Entailment
4,Prior treatment with fulvestrant or with a pho...,"["" Prior treatment with a phosphatidylinosito...",Contradiction
...,...,...,...
195,The the primary trial intervention involves on...,"[""INTERVENTION 1: "", "" Letrozole"", "" Partici...",Contradiction
196,the secondary trial reported 1 single case of ...,"[""Adverse Events 1:"", "" Total: 16/48 (33.33%)...",Entailment
197,the secondary trial and the primary trial do n...,"[""Outcome Measurement: "", "" Number of Patient...",Entailment
198,the outcome measurement of the primary trial i...,"[""Outcome Measurement: "", "" Progression-free ...",Entailment


In [9]:
hypothesis_lst=dev_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=dev_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=dev_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

200

In [10]:
dev_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

Tokenization done


In [11]:
class Model(nn.Module):
    def __init__(self,clf):
        super(Model, self).__init__()
        self.clf=clf
        self.loss=nn.CrossEntropyLoss()

    def forward(self, texts, labels, gpu=True):

        loss=self.loss(self.clf(**texts).logits, labels)

        return loss

In [12]:
model=Model(text_clf)

In [13]:
import os
import shutil
import torch
from torch import nn
from transformers import AdamW
import numpy as np
import math
import random

bat_s = 16
l_rate = 1e-5

training_data.batch_size = bat_s

model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)
total_epoch_num = 10

for epoch in range(total_epoch_num):
    training_data.on_epoch_end()
    loss_sum = 0.0
    loss_count = 0
    model.train()  # Ensure model is in training mode
    
    for batch in range(len(training_data)):
        optimizer.zero_grad()
        batch_texts, batch_labels = training_data[batch]
        loss_count += len(batch_texts['input_ids'])
        loss = model(batch_texts, batch_labels)
        print('epoch:', epoch, 'batch:', batch, 'loss:', loss.item(), end='\n' if batch == 0 or batch + 1 == len(training_data) or (batch + 1) % 1000 == 0 else '\r')
        loss_sum += loss.item() * len(batch_texts['input_ids'])
        loss.backward()
        optimizer.step()
        
    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss_sum = 0.0
    val_loss_count = 0
    with torch.no_grad():
        for batch in range(len(dev_data)):
            batch_texts, batch_labels = dev_data[batch]
            val_loss_count += len(batch_texts['input_ids'])
            val_loss = model(batch_texts, batch_labels)
            val_loss_sum += val_loss.item() * len(batch_texts['input_ids'])

    avg_val_loss = val_loss_sum / val_loss_count
    print(f'Validation - Epoch: {epoch}, Avg Loss: {avg_val_loss}')

    # Save the model after each epoch
    model_path = f'./clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{epoch:05d}.pt'
    model.clf.save_pretrained(model_path)
    print(f'Model saved to {model_path}')

_ = model.cpu()  # Move model back to CPU if necessary

epoch: 0 batch: 0 loss: 1.7612520456314087
epoch: 0 batch: 106 loss: 0.6224154233932495
Validation - Epoch: 0, Avg Loss: 0.7403052496910095
Model saved to ./clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_00000.pt
epoch: 1 batch: 0 loss: 0.6621671319007874
epoch: 1 batch: 106 loss: 0.7473458647727966
Validation - Epoch: 1, Avg Loss: 0.7227417492866516
Model saved to ./clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_00001.pt
epoch: 2 batch: 0 loss: 0.6688335537910461
epoch: 2 batch: 106 loss: 0.7268893122673035
Validation - Epoch: 2, Avg Loss: 0.7083130955696106
Model saved to ./clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_00002.pt
epoch: 3 batch: 0 loss: 0.7736801505088806
epoch: 3 batch: 106 loss: 0.5849802494049072
Validation - Epoch: 3, Avg Loss: 0.7118076968193054
Model saved to ./clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_00003.pt
epoch: 4 batch: 0 loss: 0.6448741555213928
e

In [14]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c'] + [
    '/kaggle/working/clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(dev_data)):
            batch_texts, batch_labels = dev_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [15]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.486235,0.578512,0.492958,0.70,0.490,0.492188,0.507812
1,1,0.478801,0.601626,0.506849,0.74,0.510,0.507812,0.492188
2,2,0.495646,0.605809,0.517730,0.73,0.525,0.445312,0.554688
3,3,0.514518,0.502674,0.540230,0.47,0.535,0.484375,0.515625
4,4,0.543762,0.636015,0.515528,0.83,0.525,0.492188,0.507812
5,5,0.542290,0.387097,0.545455,0.30,0.525,0.507812,0.492188
6,6,0.564929,0.610619,0.547619,0.69,0.560,0.460938,0.539062
7,7,0.599478,0.581633,0.593750,0.57,0.590,0.484375,0.515625
8,8,0.612469,0.638498,0.601770,0.68,0.615,0.453125,0.546875
9,9,0.623366,0.666667,0.536585,0.88,0.560,0.476562,0.523438


## Above results are for dev data

In [16]:
numerical_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Numerical_Statements_hypothesis_evidences.csv')
print(numerical_df)

hypothesis_lst=numerical_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=numerical_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=numerical_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

numerical_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   there is a 13.2% difference between the result...   
1   Patients with significantly elevated ejection ...   
2   Prior treatment with fulvestrant or with a pho...   
3   only patients with a HER2-positive status can ...   
4   the shortest PFS in cohort 1 of the primary tr...   
..                                                ...   
88  All the primary trial patients achieved either...   
89  A patient with a node positive T2 N2 M0 adenoc...   
90  the primary trial results show that the Trastu...   
91  the secondary trial reported 1 single case of ...   
92  All the primary trial patients had a minimum o...   

                                            evidences          label  
0   ["Outcome Measurement: ", "  Event-free Surviv...  Contradiction  
1   ["  Cardiac left ventricular function with res...  Contradiction  
2   ["  Prior treatment with a phosphatidylinosito...  Contradiction  
3   ["  HER2-positive status (p

In [17]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c'] + [
    '/kaggle/working/clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(numerical_data)):
            batch_texts, batch_labels = numerical_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [18]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.434470,0.520000,0.448276,0.619048,0.483871,0.515625,0.484375
1,1,0.442309,0.566038,0.468750,0.714286,0.505376,0.437500,0.562500
2,2,0.441177,0.574074,0.469697,0.738095,0.505376,0.531250,0.468750
3,3,0.458184,0.410959,0.483871,0.357143,0.537634,0.562500,0.437500
4,4,0.474866,0.593220,0.460526,0.833333,0.483871,0.515625,0.484375
5,5,0.473833,0.290323,0.450000,0.214286,0.526882,0.546875,0.453125
6,6,0.477711,0.531915,0.480769,0.595238,0.526882,0.546875,0.453125
7,7,0.519558,0.506329,0.540541,0.476190,0.580645,0.515625,0.484375
8,8,0.547407,0.568182,0.543478,0.595238,0.591398,0.453125,0.546875
9,9,0.544283,0.631579,0.500000,0.857143,0.548387,0.484375,0.515625


## Above results are for numerical data

In [19]:
non_numerical_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Non_Numerical_Statements_hypothesis_evidences.csv')
print(non_numerical_df)

hypothesis_lst=non_numerical_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=non_numerical_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=non_numerical_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

non_numerical_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                            hypothesis  \
0    a significant number of the participants in th...   
1    the primary trial does not report the PFS or o...   
2    the outcome measurement of the primary trial i...   
3    The the primary trial results section reports ...   
4    both cohorts of the primary trial receive iden...   
..                                                 ...   
102  Patients must be bedbound or severaly disabled...   
103  the results of the secondary trial and the pri...   
104  The the primary trial intervention involves on...   
105  the secondary trial and the primary trial do n...   
106  the outcome measurement of the primary trial i...   

                                             evidences          label  
0    ["  Enterocolitis 1/167 (0.60%)", "  Enterocol...  Contradiction  
1    ["Outcome Measurement: ", "  Local Control Usi...     Entailment  
2    ["Outcome Measurement: ", "  Progression-free ...  Contradiction  
3    ["Outcome 

In [20]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c'] + [
    '/kaggle/working/clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(non_numerical_data)):
            batch_texts, batch_labels = non_numerical_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [21]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.549065,0.619718,0.523810,0.758621,0.495327,0.520000,0.480000
1,1,0.513855,0.628571,0.536585,0.758621,0.514019,0.493333,0.506667
2,2,0.538427,0.631579,0.560000,0.724138,0.542056,0.466667,0.533333
3,3,0.557226,0.561404,0.571429,0.551724,0.532710,0.480000,0.520000
4,4,0.594571,0.671329,0.564706,0.827586,0.560748,0.413333,0.586667
5,5,0.597352,0.451613,0.600000,0.362069,0.523364,0.493333,0.506667
6,6,0.639890,0.666667,0.594595,0.758621,0.588785,0.440000,0.560000
7,7,0.656763,0.632479,0.627119,0.637931,0.598131,0.360000,0.640000
8,8,0.659471,0.688000,0.641791,0.741379,0.635514,0.373333,0.626667
9,9,0.685695,0.693333,0.565217,0.896552,0.570093,0.440000,0.560000


## Above results are for Non numerical data

In [22]:
single_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Single_hypothesis_evidences.csv')
print(single_df)

hypothesis_lst=single_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=single_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=single_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

single_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                            hypothesis  \
0    there is a 13.2% difference between the result...   
1    the primary trial does not report the PFS or o...   
2    Prior treatment with fulvestrant or with a pho...   
3    only patients with a HER2-positive status can ...   
4    the shortest PFS in cohort 1 of the primary tr...   
..                                                 ...   
135  A patient with a node positive T2 N2 M0 adenoc...   
136  Patients must be bedbound or severaly disabled...   
137  the primary trial results show that the Trastu...   
138  the outcome measurement of the primary trial i...   
139  All the primary trial patients had a minimum o...   

                                             evidences          label  
0    ["Outcome Measurement: ", "  Event-free Surviv...  Contradiction  
1    ["Outcome Measurement: ", "  Local Control Usi...     Entailment  
2    ["  Prior treatment with a phosphatidylinosito...  Contradiction  
3    ["  HER2-p

In [23]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c'] + [
    '/kaggle/working/clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(single_data)):
            batch_texts, batch_labels = single_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [24]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.488902,0.574850,0.494845,0.685714,0.492857,0.458333,0.541667
1,1,0.471385,0.581395,0.490196,0.714286,0.485714,0.406250,0.593750
2,2,0.483836,0.626374,0.508929,0.814286,0.514286,0.385417,0.614583
3,3,0.502552,0.541667,0.527027,0.557143,0.528571,0.500000,0.500000
4,4,0.520892,0.631016,0.504274,0.842857,0.507143,0.427083,0.572917
5,5,0.517057,0.423729,0.520833,0.357143,0.514286,0.562500,0.437500
6,6,0.522979,0.585366,0.510638,0.685714,0.514286,0.458333,0.541667
7,7,0.562060,0.573427,0.561644,0.585714,0.564286,0.468750,0.531250
8,8,0.569028,0.619355,0.564706,0.685714,0.578571,0.437500,0.562500
9,9,0.582258,0.655556,0.536364,0.842857,0.557143,0.395833,0.604167


## Above results are for Single Type data

In [25]:
comparison_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Comparison_hypothesis_evidences.csv')
print(comparison_df)

hypothesis_lst=comparison_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=comparison_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=comparison_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

comparison_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   Patients with significantly elevated ejection ...   
1   a significant number of the participants in th...   
2   Diarrhoea is the most common adverse event in ...   
3   most participants in the secondary trial and t...   
4   Presence of hot flashes for at least 4 month p...   
5   the secondary trial and the primary trial both...   
6   the primary trial and the secondary trial use ...   
7   the primary trial and the secondary trial do n...   
8   the secondary trial and the primary trial do n...   
9   Between the secondary trial and the primary tr...   
10  All patients in the primary trial receive high...   
11  all participants of the primary trial have sta...   
12  The results from the secondary trial and the p...   
13  the primary trial and the secondary trial part...   
14  There are more cases of Urosepsis in the prima...   
15  the primary trial participants are not adminis...   
16  All the primary trial parti

In [26]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c'] + [
    '/kaggle/working/clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(comparison_data)):
            batch_texts, batch_labels = comparison_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [27]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.500698,0.586667,0.488889,0.733333,0.483333,0.516667,0.483333
1,1,0.524467,0.648649,0.545455,0.800000,0.566667,0.433333,0.566667
2,2,0.573564,0.542373,0.551724,0.533333,0.550000,0.450000,0.550000
3,3,0.592130,0.372093,0.615385,0.266667,0.550000,0.450000,0.550000
4,4,0.645147,0.648649,0.545455,0.800000,0.566667,0.433333,0.566667
5,5,0.694160,0.270270,0.714286,0.166667,0.550000,0.450000,0.550000
6,6,0.749025,0.677419,0.656250,0.700000,0.666667,0.333333,0.666667
7,7,0.709124,0.603774,0.695652,0.533333,0.650000,0.350000,0.650000
8,8,0.748853,0.689655,0.714286,0.666667,0.700000,0.300000,0.700000
9,9,0.711531,0.690476,0.537037,0.966667,0.566667,0.433333,0.566667


## Above results are for Comparison Type data

In [28]:
adverse_events_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/AdverseEvents_hypothesis_evidences.csv')
print(adverse_events_df)

hypothesis_lst=adverse_events_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=adverse_events_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=adverse_events_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

adverse_events_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   a significant number of the participants in th...   
1   The only case of congestive heart failure in t...   
2   Anaemia was the most common adverse event in c...   
3   The adverse events section in the primary tria...   
4   Diarrhoea is the most common adverse event in ...   
5   most participants in the secondary trial and t...   
6   The adverse events section in the primary tria...   
7   there were 4 types of Adverse events which did...   
8   None of the individual Aes recorded in the pri...   
9   The most common adverse event in the primary t...   
10  Febrile Neutropenia was the most common advers...   
11  None of the individual Aes recorded in the pri...   
12  the primary trial and the secondary trial do n...   
13  Between the secondary trial and the primary tr...   
14  Syncope was the most common adverse event reco...   
15  no cardiac or neural adverse events were recor...   
16  there was a higher percenta

In [29]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c'] + [
    '/kaggle/working/clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(adverse_events_data)):
            batch_texts, batch_labels = adverse_events_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [30]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.481918,0.590164,0.514286,0.692308,0.519231,0.480769,0.519231
1,1,0.503862,0.637681,0.511628,0.846154,0.519231,0.480769,0.519231
2,2,0.507371,0.600000,0.529412,0.692308,0.538462,0.461538,0.538462
3,3,0.540595,0.489796,0.521739,0.461538,0.519231,0.480769,0.519231
4,4,0.569779,0.619718,0.488889,0.846154,0.480769,0.519231,0.480769
5,5,0.576932,0.421053,0.666667,0.307692,0.576923,0.423077,0.576923
6,6,0.573015,0.606061,0.500000,0.769231,0.500000,0.500000,0.500000
7,7,0.585835,0.489796,0.521739,0.461538,0.519231,0.480769,0.519231
8,8,0.601927,0.633333,0.558824,0.730769,0.576923,0.423077,0.576923
9,9,0.629663,0.686567,0.560976,0.884615,0.596154,0.403846,0.596154


## Above results are for Adverse events data

In [31]:
results_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Results_hypothesis_evidences.csv')
print(results_df)

hypothesis_lst=results_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=results_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=results_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

results_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   there is a 13.2% difference between the result...   
1   the primary trial does not report the PFS or o...   
2   the shortest PFS in cohort 1 of the primary tr...   
3   the outcome measurement of the primary trial i...   
4   The the primary trial results section reports ...   
5   The the primary trial results section reports ...   
6   The shortest PFS in cohort 1 of the primary tr...   
7   the secondary trial and the primary trial both...   
8   there is a minimal difference between the resu...   
9   the primary trial and the secondary trial use ...   
10  at least 2 participants of the primary trial, ...   
11  the primary trial does not report the PFS or o...   
12  The results from the secondary trial and the p...   
13  neither of the interventions in the primary tr...   
14  The shortest PFS in cohort 1 of the primary tr...   
15  there were 6 recorded deaths in cohort 1 of th...   
16  the primary trial reports t

In [32]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c'] + [
    '/kaggle/working/clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(results_data)):
            batch_texts, batch_labels = results_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [33]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.510960,0.524590,0.484848,0.571429,0.482143,0.517857,0.482143
1,1,0.533342,0.657895,0.520833,0.892857,0.535714,0.464286,0.535714
2,2,0.546945,0.571429,0.514286,0.642857,0.517857,0.482143,0.517857
3,3,0.559653,0.444444,0.588235,0.357143,0.553571,0.446429,0.553571
4,4,0.592571,0.676471,0.575000,0.821429,0.607143,0.392857,0.607143
5,5,0.600626,0.300000,0.500000,0.214286,0.500000,0.500000,0.500000
6,6,0.650917,0.596491,0.586207,0.607143,0.589286,0.410714,0.589286
7,7,0.675175,0.631579,0.620690,0.642857,0.625000,0.375000,0.625000
8,8,0.705949,0.655172,0.633333,0.678571,0.642857,0.357143,0.642857
9,9,0.686806,0.675325,0.530612,0.928571,0.553571,0.446429,0.553571


## Above results are for Results data

In [34]:
eligibility_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Eligibility_hypothesis_evidences.csv')
print(eligibility_df)

hypothesis_lst=eligibility_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=eligibility_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=eligibility_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

eligibility_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   Patients with significantly elevated ejection ...   
1   Prior treatment with fulvestrant or with a pho...   
2   only patients with a HER2-positive status can ...   
3   T4 N2 M4 patients are eligible for the primary...   
4   Patients must have a palpable carcinoma to be ...   
5   to be eligible for the primary trial patients ...   
6   ongoing flucytosine treatments are not permitt...   
7   Only patients with triple negative breast canc...   
8   adults and children can take part in the prima...   
9   Prior treatment with fulvestrant or with a pho...   
10  Presence of hot flashes for at least 4 month p...   
11  to be eligible for the primary trial candidate...   
12  A patient with a node positive T2 N2 M0 adenoc...   
13  ECOG score < 2 is necessary to be eligible for...   
14  patients with a Malignant brain tumour or colo...   
15  the secondary trial and the primary trial do n...   
16  all participants of the pri

In [35]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c'] + [
    '/kaggle/working/clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(eligibility_data)):
            batch_texts, batch_labels = eligibility_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [36]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.495480,0.602740,0.488889,0.785714,0.482143,0.517857,0.482143
1,1,0.469188,0.472727,0.481481,0.464286,0.482143,0.517857,0.482143
2,2,0.487548,0.597015,0.512821,0.714286,0.517857,0.482143,0.517857
3,3,0.489454,0.509091,0.518519,0.500000,0.517857,0.482143,0.517857
4,4,0.489156,0.600000,0.500000,0.750000,0.500000,0.500000,0.500000
5,5,0.494503,0.408163,0.476190,0.357143,0.482143,0.517857,0.482143
6,6,0.521849,0.586207,0.566667,0.607143,0.571429,0.428571,0.571429
7,7,0.567375,0.500000,0.541667,0.464286,0.535714,0.464286,0.535714
8,8,0.595145,0.555556,0.576923,0.535714,0.571429,0.428571,0.571429
9,9,0.600822,0.617647,0.525000,0.750000,0.535714,0.464286,0.535714


## Above results are for Eligibility data

In [37]:
intervention_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Intervention_hypothesis_evidences.csv')
print(intervention_df)

hypothesis_lst=intervention_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=intervention_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=intervention_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

intervention_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   Both cohorts of the primary trial undergo a to...   
1   both cohorts of the primary trial receive iden...   
2   the primary trial participants recieve doses o...   
3   Cohort 1 and 2 of the primary trial do not rec...   
4   Participants of the primary trial will not rec...   
5   the primary trial participants receive more th...   
6   All patients in the primary trial receive high...   
7   the primary trial participants are administere...   
8   the primary trial and the secondary trial part...   
9   Cohort 1 and 2 of the primary trial do not rec...   
10  the primary trial participants are not adminis...   
11  All the primary trial participants are adminis...   
12  the primary trial participants are administere...   
13  All the primary trial participants are adminis...   
14  the primary trial participants are administere...   
15  80% of patients in the primary trial received ...   
16  the primary trial participa

In [38]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c'] + [
    '/kaggle/working/clf_models/MoritzLaurer/DeBERTa-v3-small-mnli-fever-docnli-ling-2c_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(intervention_data)):
            batch_texts, batch_labels = intervention_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

In [39]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.539900,0.595745,0.482759,0.777778,0.472222,0.527778,0.472222
1,1,0.520176,0.608696,0.500000,0.777778,0.500000,0.500000,0.500000
2,2,0.558075,0.666667,0.515152,0.944444,0.527778,0.472222,0.527778
3,3,0.575225,0.578947,0.550000,0.611111,0.555556,0.444444,0.555556
4,4,0.558473,0.653846,0.500000,0.944444,0.500000,0.500000,0.500000
5,5,0.589380,0.428571,0.600000,0.333333,0.555556,0.444444,0.555556
6,6,0.605677,0.666667,0.555556,0.833333,0.583333,0.416667,0.583333
7,7,0.619440,0.736842,0.700000,0.777778,0.722222,0.277778,0.722222
8,8,0.650103,0.731707,0.652174,0.833333,0.694444,0.305556,0.694444
9,9,0.619323,0.692308,0.529412,1.000000,0.555556,0.444444,0.555556


## Above results are for Intervention data